# 🔬 High-N Topological Investigation (N=20k)

**Objective**: Investigate spectral topology at high resolution

**NOT formalizing** — just exploring:
1. Convergence behavior at N=20k
2. Zero-mode structure in 1-form spectrum
3. Mode localization patterns
4. Graph topology vs TCS ratio

**Memory management**: Chunked computation where needed

In [ ]:
# =============================================================================
# SETUP WITH MEMORY MONITORING
# =============================================================================

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, lil_matrix, diags, eye
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import cdist
import gc
import time
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

def get_memory_mb():
    """Get current memory usage in MB."""
    import psutil
    return psutil.Process().memory_info().rss / 1024**2

def log_memory(msg=""):
    print(f"  [MEM] {get_memory_mb():.0f} MB {msg}")

plt.style.use('seaborn-v0_8-whitegrid')

print(f"✓ Setup complete - {datetime.now().strftime('%Y-%m-%d %H:%M')}")
log_memory("at start")

In [ ]:
# =============================================================================
# MEMORY-EFFICIENT SAMPLING
# =============================================================================

def sample_S3(n, rng):
    """Sample uniformly from S³ (float32 for memory)."""
    q = rng.standard_normal((n, 4)).astype(np.float32)
    return q / np.linalg.norm(q, axis=1, keepdims=True)

def sample_TCS(n, rng):
    """Sample from TCS = S¹ × S³ × S³."""
    theta = rng.uniform(0, 2*np.pi, n).astype(np.float32)
    q1 = sample_S3(n, rng)
    q2 = sample_S3(n, rng)
    return theta, q1, q2

print("✓ Sampling functions loaded")

In [ ]:
# =============================================================================
# CHUNKED DISTANCE COMPUTATION (Memory-efficient)
# =============================================================================

def geodesic_S1_chunk(theta1, theta2):
    """Geodesic distance on S¹ between two sets."""
    diff = np.abs(theta1[:, None] - theta2[None, :])
    return np.minimum(diff, 2*np.pi - diff).astype(np.float32)

def geodesic_S3_chunk(Q1, Q2):
    """Geodesic distance on S³ between two sets."""
    dot = np.clip(np.abs(Q1 @ Q2.T), 0, 1)
    return (2 * np.arccos(dot)).astype(np.float32)

def compute_tcs_distance_chunked(
    theta, q1, q2,
    ratio=1.0,
    det_g=65/32,
    chunk_size=2000
):
    """
    Compute TCS distance matrix in chunks to manage memory.
    Returns SPARSE k-NN graph directly (not full distance matrix).
    """
    n = len(theta)
    alpha = det_g / (ratio**3)

    print(f"  Computing chunked distances (n={n}, chunk={chunk_size})...")

    # We'll build a sparse k-NN graph directly
    # First pass: compute all distances in chunks and find k-NN
    k = 50  # k for k-NN
    
    # Store k nearest neighbors for each point
    knn_indices = np.zeros((n, k), dtype=np.int32)
    knn_distances = np.zeros((n, k), dtype=np.float32)

    n_chunks = (n + chunk_size - 1) // chunk_size

    for i in range(n_chunks):
        i_start = i * chunk_size
        i_end = min((i + 1) * chunk_size, n)

        # Get chunk data
        theta_i = theta[i_start:i_end]
        q1_i = q1[i_start:i_end]
        q2_i = q2[i_start:i_end]

        # Compute distances to ALL other points
        d_s1 = geodesic_S1_chunk(theta_i, theta)
        d_s3_1 = geodesic_S3_chunk(q1_i, q1)
        d_s3_2 = geodesic_S3_chunk(q2_i, q2)

        # TCS metric
        D_chunk = np.sqrt(alpha * d_s1**2 + d_s3_1**2 + (ratio**2) * d_s3_2**2)

        # Set self-distance to inf
        for j in range(i_end - i_start):
            D_chunk[j, i_start + j] = np.inf

        # Find k nearest neighbors for this chunk
        for j in range(i_end - i_start):
            idx = np.argpartition(D_chunk[j], k)[:k]
            knn_indices[i_start + j] = idx
            knn_distances[i_start + j] = D_chunk[j, idx]

        # Clear memory
        del d_s1, d_s3_1, d_s3_2, D_chunk
        gc.collect()

        if (i + 1) % 5 == 0:
            print(f"    Chunk {i+1}/{n_chunks} done")

    return knn_indices, knn_distances

print("✓ Chunked distance computation loaded")

In [ ]:
# =============================================================================
# SPARSE GRAPH LAPLACIAN FROM k-NN
# =============================================================================

def build_sparse_laplacian(knn_indices, knn_distances, sigma=None):
    """
    Build sparse normalized Laplacian from k-NN data.
    """
    n = knn_indices.shape[0]
    k = knn_indices.shape[1]

    # Compute sigma from median k-NN distance
    if sigma is None:
        sigma = float(np.median(knn_distances))
    print(f"  Using σ = {sigma:.6f}")

    # Build sparse weight matrix
    W = lil_matrix((n, n), dtype=np.float32)

    for i in range(n):
        for j_idx in range(k):
            j = knn_indices[i, j_idx]
            d = knn_distances[i, j_idx]
            w = np.exp(-d**2 / (2 * sigma**2))
            W[i, j] = w
            W[j, i] = w  # Symmetrize

    W = W.tocsr()

    # Degree
    d = np.array(W.sum(axis=1)).flatten()
    d = np.maximum(d, 1e-10)
    d_inv_sqrt = 1.0 / np.sqrt(d)

    # Normalized Laplacian: L = I - D^{-1/2} W D^{-1/2}
    D_inv_sqrt = diags(d_inv_sqrt)
    L = eye(n) - D_inv_sqrt @ W @ D_inv_sqrt

    return L.tocsr(), W, sigma, d

print("✓ Sparse Laplacian builder loaded")

In [ ]:
# =============================================================================
# TOPOLOGICAL ANALYSIS FUNCTIONS
# =============================================================================

def count_connected_components(W):
    """Count connected components using BFS."""
    from scipy.sparse.csgraph import connected_components
    n_components, labels = connected_components(W, directed=False)
    return n_components, labels

def estimate_betti_1(W, n_vertices):
    """
    Estimate first Betti number (number of independent cycles).
    β₁ = m - n + c  (Euler formula for graphs)
    where m = edges, n = vertices, c = components
    """
    n_edges = W.nnz // 2  # Symmetric, so divide by 2
    n_components, _ = count_connected_components(W)
    beta_1 = n_edges - n_vertices + n_components
    return beta_1, n_edges, n_components

def count_triangles_sparse(W, sample_size=5000):
    """
    Estimate number of triangles by sampling.
    (Full count is O(n³) which is too expensive)
    """
    n = W.shape[0]
    W_binary = (W > 0).astype(np.float32)

    # Sample random triplets
    rng = np.random.default_rng(42)
    triangles_found = 0

    indices = rng.choice(n, size=min(sample_size, n), replace=False)

    for i in indices:
        neighbors_i = W_binary[i].nonzero()[1]
        if len(neighbors_i) < 2:
            continue
        # Check pairs of neighbors
        for j_idx in range(min(20, len(neighbors_i))):
            j = neighbors_i[j_idx]
            neighbors_j = set(W_binary[j].nonzero()[1])
            # Count common neighbors
            common = len(set(neighbors_i) & neighbors_j)
            triangles_found += common

    # Scale estimate
    scale_factor = n / len(indices)
    estimated_triangles = int(triangles_found * scale_factor / 6)  # Each triangle counted 6 times

    return estimated_triangles

print("✓ Topological analysis functions loaded")

In [ ]:
# =============================================================================
# 1-FORM HODGE LAPLACIAN (Sparse version)
# =============================================================================

def build_hodge_1_sparse(W, max_edges=50000):
    """
    Build sparse Hodge 1-form Laplacian.
    Limited to max_edges for memory.
    """
    n = W.shape[0]

    # Extract edges
    W_coo = W.tocoo()
    edges = []
    for i, j, w in zip(W_coo.row, W_coo.col, W_coo.data):
        if i < j and w > 0:  # Upper triangular only
            edges.append((i, j, w))

    m = len(edges)
    print(f"  Graph has {m} edges")

    if m > max_edges:
        print(f"  WARNING: Limiting to {max_edges} strongest edges")
        edges = sorted(edges, key=lambda x: -x[2])[:max_edges]
        m = max_edges

    # Build incidence matrix d₀: vertices → edges
    d0 = lil_matrix((m, n), dtype=np.float32)
    edge_weights = np.zeros(m, dtype=np.float32)

    for e_idx, (i, j, w) in enumerate(edges):
        d0[e_idx, i] = -1.0
        d0[e_idx, j] = +1.0
        edge_weights[e_idx] = w

    d0 = d0.tocsr()

    # Hodge 1-Laplacian: Δ₁ = d₀ᵀ d₀
    L1 = d0 @ d0.T

    # Normalize by edge degree
    d_edge = np.array(np.abs(L1).sum(axis=1)).flatten()
    d_edge = np.maximum(d_edge, 1e-10)
    d_inv_sqrt = 1.0 / np.sqrt(d_edge)
    D_inv = diags(d_inv_sqrt)
    L1_norm = D_inv @ L1 @ D_inv

    return L1_norm.tocsr(), m, edges

print("✓ Sparse Hodge 1-form Laplacian loaded")

In [ ]:
# =============================================================================
# MODE ANALYSIS FUNCTIONS
# =============================================================================

def participation_ratio(v):
    """PR = 1/(N × Σ|vᵢ|⁴)."""
    v = v.flatten()
    v_norm = v / np.linalg.norm(v)
    return float(1.0 / (len(v) * np.sum(v_norm**4)))

def mode_projection_stats(v, theta, q1, q2):
    """
    Compute statistics of mode projection onto each factor.
    """
    v = v.flatten()
    p = np.abs(v)**2
    p = p / np.sum(p)

    # S¹ statistics
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    s1_cos_moment = np.sum(p * cos_theta)
    s1_sin_moment = np.sum(p * sin_theta)
    s1_order_param = np.sqrt(s1_cos_moment**2 + s1_sin_moment**2)

    # S³ statistics (use first coordinate as proxy)
    s3_1_mean = np.sum(p[:, None] * q1, axis=0)
    s3_1_order = np.linalg.norm(s3_1_mean)

    s3_2_mean = np.sum(p[:, None] * q2, axis=0)
    s3_2_order = np.linalg.norm(s3_2_mean)

    # Fourier-like analysis on S¹
    # Mode 1: cos(θ), sin(θ)
    fourier_1 = np.sqrt((np.sum(p * np.cos(theta)))**2 + (np.sum(p * np.sin(theta)))**2)
    # Mode 2: cos(2θ), sin(2θ)
    fourier_2 = np.sqrt((np.sum(p * np.cos(2*theta)))**2 + (np.sum(p * np.sin(2*theta)))**2)

    return {
        "s1_order": float(s1_order_param),
        "s3_1_order": float(s3_1_order),
        "s3_2_order": float(s3_2_order),
        "s1_fourier_1": float(fourier_1),
        "s1_fourier_2": float(fourier_2)
    }

print("✓ Mode analysis functions loaded")

---

# 🔬 Main Investigation

In [ ]:
# =============================================================================
# FULL ANALYSIS PIPELINE
# =============================================================================

def full_analysis(N, ratio, seed=42, H_star=99):
    """
    Complete analysis pipeline for a single (N, ratio) configuration.
    """
    print(f"\n{'='*60}")
    print(f"  N={N}, ratio={ratio:.2f}")
    print(f"{'='*60}")

    t0 = time.time()
    log_memory("start")

    # 1. Sample TCS
    print("\n[1] Sampling TCS...")
    rng = np.random.default_rng(seed)
    theta, q1, q2 = sample_TCS(N, rng)
    log_memory("after sampling")

    # 2. Compute k-NN distances (chunked)
    print("\n[2] Computing k-NN distances...")
    chunk_size = 2000 if N > 10000 else N
    knn_idx, knn_dist = compute_tcs_distance_chunked(
        theta, q1, q2, ratio=ratio, chunk_size=chunk_size
    )
    log_memory("after k-NN")

    # 3. Build sparse Laplacian
    print("\n[3] Building sparse Laplacian...")
    L, W, sigma, degrees = build_sparse_laplacian(knn_idx, knn_dist)
    del knn_idx, knn_dist
    gc.collect()
    log_memory("after Laplacian")

    # 4. Compute eigenvalues (0-form)
    print("\n[4] Computing 0-form spectrum...")
    n_eig = 20
    eigenvalues, eigenvectors = eigsh(L, k=n_eig, which='SM', tol=1e-8)
    idx = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # First non-zero
    mu1 = 0.0
    v1_idx = 1
    for i, ev in enumerate(eigenvalues):
        if ev > 1e-8:
            mu1 = float(ev)
            v1_idx = i
            break

    v1 = eigenvectors[:, v1_idx]
    lambda1_hat = mu1 / (sigma**2)

    print(f"  μ₁ = {mu1:.6f}")
    print(f"  σ = {sigma:.6f}")
    print(f"  λ̂₁ = μ₁/σ² = {lambda1_hat:.6f}")
    print(f"  λ̂₁ × H* = {lambda1_hat * H_star:.4f}")

    # 5. Mode analysis
    print("\n[5] Analyzing mode structure...")
    pr = participation_ratio(v1)
    mode_stats = mode_projection_stats(v1, theta, q1, q2)
    print(f"  PR = {pr:.4f}")
    print(f"  S¹ order = {mode_stats['s1_order']:.4f}")
    print(f"  S¹ Fourier(1) = {mode_stats['s1_fourier_1']:.4f}")
    print(f"  S¹ Fourier(2) = {mode_stats['s1_fourier_2']:.4f}")

    # 6. Topological analysis
    print("\n[6] Analyzing graph topology...")
    beta_1, n_edges, n_components = estimate_betti_1(W, N)
    print(f"  Vertices: {N}")
    print(f"  Edges: {n_edges}")
    print(f"  Components: {n_components}")
    print(f"  β₁ (cycles): {beta_1}")
    print(f"  Avg degree: {2*n_edges/N:.1f}")

    n_triangles = count_triangles_sparse(W)
    print(f"  Est. triangles: ~{n_triangles}")

    # 7. 1-form spectrum
    print("\n[7] Computing 1-form spectrum...")
    L1, m_edges, edges = build_hodge_1_sparse(W, max_edges=40000)

    try:
        n_eig_1 = min(50, m_edges - 1)
        eig1, _ = eigsh(L1, k=n_eig_1, which='SM', tol=1e-6)
        eig1 = np.sort(eig1)

        # Count zero modes
        zero_threshold = 1e-5
        n_zero_modes = np.sum(eig1 < zero_threshold)
        first_nonzero_1 = 0.0
        for ev in eig1:
            if ev > zero_threshold:
                first_nonzero_1 = float(ev)
                break

        print(f"  Zero modes (< {zero_threshold}): {n_zero_modes}/{n_eig_1}")
        print(f"  First non-zero λ₁(Δ₁): {first_nonzero_1:.6f}")
        print(f"  λ₁(Δ₁)/σ² = {first_nonzero_1/(sigma**2):.6f}")

    except Exception as e:
        print(f"  1-form computation failed: {e}")
        eig1 = np.array([])
        n_zero_modes = -1
        first_nonzero_1 = np.nan

    elapsed = time.time() - t0
    print(f"\n  Total time: {elapsed:.1f}s")
    log_memory("end")

    # Cleanup
    del L, W, L1, eigenvectors
    gc.collect()

    return {
        "N": N,
        "ratio": ratio,
        "sigma": sigma,
        "mu1": mu1,
        "lambda1_hat": lambda1_hat,
        "product": lambda1_hat * H_star,
        "PR": pr,
        "mode_stats": mode_stats,
        "n_edges": n_edges,
        "n_components": n_components,
        "beta_1": beta_1,
        "n_triangles": n_triangles,
        "eigenvalues_0": eigenvalues[:10].tolist(),
        "eigenvalues_1": eig1[:20].tolist() if len(eig1) > 0 else [],
        "n_zero_modes_1form": n_zero_modes,
        "lambda1_1form": first_nonzero_1,
        "elapsed": elapsed
    }

print("✓ Analysis pipeline ready")

In [ ]:
# =============================================================================
# RUN INVESTIGATION AT N=20000
# =============================================================================

print("\n" + "#"*70)
print("#  HIGH-N TOPOLOGICAL INVESTIGATION (N=20,000)")
print("#"*70)

N = 20000
ratios_to_test = [1.0, 1.18, 1.3, 1.4, 1.6]

all_results = []

for ratio in ratios_to_test:
    result = full_analysis(N=N, ratio=ratio, seed=42)
    all_results.append(result)
    gc.collect()  # Clean up between runs

In [ ]:
# =============================================================================
# CONVERGENCE TEST: COMPARE N=5k, 10k, 20k at ratio=1.18
# =============================================================================

print("\n" + "#"*70)
print("#  CONVERGENCE TEST: N = 5k, 10k, 20k at ratio=1.18")
print("#"*70)

convergence_results = []

for N_test in [5000, 10000, 20000]:
    result = full_analysis(N=N_test, ratio=1.18, seed=42)
    convergence_results.append(result)
    gc.collect()

In [ ]:
# =============================================================================
# SUMMARY TABLE
# =============================================================================

print("\n" + "="*80)
print("  RATIO SWEEP SUMMARY (N=20,000)")
print("="*80)

print(f"\n{'Ratio':>6} | {'λ̂₁×H*':>8} | {'PR':>6} | {'β₁':>7} | {'#Edges':>7} | {'Zero(Δ₁)':>9} | {'S¹ F(1)':>8}")
print("-"*75)

for r in all_results:
    print(f"{r['ratio']:6.2f} | {r['product']:8.4f} | {r['PR']:6.3f} | "
          f"{r['beta_1']:7d} | {r['n_edges']:7d} | {r['n_zero_modes_1form']:9d} | "
          f"{r['mode_stats']['s1_fourier_1']:8.4f}")

In [ ]:
# =============================================================================
# CONVERGENCE SUMMARY
# =============================================================================

print("\n" + "="*80)
print("  CONVERGENCE SUMMARY (ratio=1.18)")
print("="*80)

print(f"\n{'N':>7} | {'λ̂₁×H*':>10} | {'σ':>10} | {'μ₁':>10} | {'β₁':>8} | {'PR':>6}")
print("-"*65)

for r in convergence_results:
    print(f"{r['N']:7d} | {r['product']:10.4f} | {r['sigma']:10.6f} | "
          f"{r['mu1']:10.6f} | {r['beta_1']:8d} | {r['PR']:6.3f}")

# Extrapolation
if len(convergence_results) >= 3:
    Ns = np.array([r['N'] for r in convergence_results])
    prods = np.array([r['product'] for r in convergence_results])

    # Fit: product = a + b/sqrt(N)
    X = 1.0 / np.sqrt(Ns)
    coeffs = np.polyfit(X, prods, 1)
    product_inf = coeffs[1]  # Intercept = limit as N→∞

    print(f"\n  Linear fit: λ̂₁×H* = {coeffs[1]:.4f} + {coeffs[0]:.4f}/√N")
    print(f"  Extrapolated limit (N→∞): λ̂₁×H* → {product_inf:.4f}")

In [ ]:
# =============================================================================
# VISUALIZATION: TOPOLOGY VS RATIO
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

ratios_plot = [r['ratio'] for r in all_results]
products = [r['product'] for r in all_results]
prs = [r['PR'] for r in all_results]
beta1s = [r['beta_1'] for r in all_results]
zero_modes = [r['n_zero_modes_1form'] for r in all_results]
s1_fourier = [r['mode_stats']['s1_fourier_1'] for r in all_results]
n_edges = [r['n_edges'] for r in all_results]

# 1. Spectral product
ax = axes[0, 0]
ax.plot(ratios_plot, products, 'o-', markersize=10, linewidth=2)
ax.axhline(y=13, color='red', linestyle='--', alpha=0.7, label='13')
ax.axhline(y=21, color='blue', linestyle='--', alpha=0.7, label='21')
ax.set_xlabel('Ratio', fontsize=12)
ax.set_ylabel('λ̂₁ × H*', fontsize=12)
ax.set_title('Spectral Product (N=20k)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Participation ratio
ax = axes[0, 1]
ax.plot(ratios_plot, prs, 's-', markersize=10, linewidth=2, color='purple')
ax.set_xlabel('Ratio', fontsize=12)
ax.set_ylabel('Participation Ratio', fontsize=12)
ax.set_title('Mode Delocalization', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# 3. β₁ (cycles)
ax = axes[0, 2]
ax.plot(ratios_plot, beta1s, 'D-', markersize=10, linewidth=2, color='green')
ax.set_xlabel('Ratio', fontsize=12)
ax.set_ylabel('β₁ (graph cycles)', fontsize=12)
ax.set_title('First Betti Number of Graph', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# 4. Zero modes (1-form)
ax = axes[1, 0]
ax.plot(ratios_plot, zero_modes, '^-', markersize=10, linewidth=2, color='orange')
ax.set_xlabel('Ratio', fontsize=12)
ax.set_ylabel('Zero modes in Δ₁', fontsize=12)
ax.set_title('1-Form Harmonic Modes', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# 5. S¹ Fourier(1) component
ax = axes[1, 1]
ax.plot(ratios_plot, s1_fourier, 'v-', markersize=10, linewidth=2, color='brown')
ax.set_xlabel('Ratio', fontsize=12)
ax.set_ylabel('S¹ Fourier(1) amplitude', fontsize=12)
ax.set_title('Mode S¹ Structure (cos θ)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# 6. Convergence
ax = axes[1, 2]
Ns_conv = [r['N'] for r in convergence_results]
prods_conv = [r['product'] for r in convergence_results]
ax.plot(Ns_conv, prods_conv, 'o-', markersize=12, linewidth=2, color='darkblue')
ax.axhline(y=13, color='red', linestyle='--', alpha=0.7)
if 'product_inf' in dir():
    ax.axhline(y=product_inf, color='green', linestyle=':', label=f'Extrap: {product_inf:.2f}')
ax.set_xlabel('N (sample size)', fontsize=12)
ax.set_ylabel('λ̂₁ × H*', fontsize=12)
ax.set_title('Convergence (ratio=1.18)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('high_n_topology_investigation.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved: high_n_topology_investigation.png")

In [ ]:
# =============================================================================
# DETAILED MODE STRUCTURE VISUALIZATION
# =============================================================================

# Re-run at ratio=1.18 to get eigenvector for plotting
print("Re-computing at ratio=1.18 for mode visualization...")

N_viz = 10000  # Smaller for visualization
rng = np.random.default_rng(42)
theta, q1, q2 = sample_TCS(N_viz, rng)

knn_idx, knn_dist = compute_tcs_distance_chunked(theta, q1, q2, ratio=1.18, chunk_size=2000)
L, W, sigma, _ = build_sparse_laplacian(knn_idx, knn_dist)
eigenvalues, eigenvectors = eigsh(L, k=10, which='SM', tol=1e-8)
idx = np.argsort(eigenvalues)
v1 = eigenvectors[:, idx[1]]  # First non-constant mode

# Normalize for plotting
v1_plot = np.abs(v1)
v1_plot = v1_plot / np.max(v1_plot)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# S¹ projection
ax = axes[0]
ax.scatter(theta, v1_plot, c=v1_plot, cmap='viridis', s=1, alpha=0.5)
ax.set_xlabel('θ (S¹ angle)', fontsize=12)
ax.set_ylabel('|v₁|', fontsize=12)
ax.set_title('Mode on S¹ (ratio=1.18)', fontsize=14, fontweight='bold')

# S³₁ projection (first coordinate)
ax = axes[1]
ax.scatter(q1[:, 0], v1_plot, c=v1_plot, cmap='viridis', s=1, alpha=0.5)
ax.set_xlabel('q₁[0] (S³₁ coord)', fontsize=12)
ax.set_ylabel('|v₁|', fontsize=12)
ax.set_title('Mode on S³₁', fontsize=14, fontweight='bold')

# S³₂ projection
ax = axes[2]
ax.scatter(q2[:, 0], v1_plot, c=v1_plot, cmap='viridis', s=1, alpha=0.5)
ax.set_xlabel('q₂[0] (S³₂ coord)', fontsize=12)
ax.set_ylabel('|v₁|', fontsize=12)
ax.set_title('Mode on S³₂', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('mode_structure_ratio118.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved: mode_structure_ratio118.png")

In [ ]:
# =============================================================================
# EXPORT ALL RESULTS
# =============================================================================

export_data = {
    "timestamp": datetime.now().isoformat(),
    "investigation": "high_n_topology",
    "ratio_sweep": all_results,
    "convergence_test": convergence_results,
    "extrapolated_limit": float(product_inf) if 'product_inf' in dir() else None
}

with open('high_n_topology_results.json', 'w') as f:
    json.dump(export_data, f, indent=2, default=str)

print("\n" + "="*70)
print("  INVESTIGATION COMPLETE")
print("="*70)
print("\nExported files:")
print("  - high_n_topology_results.json")
print("  - high_n_topology_investigation.png")
print("  - mode_structure_ratio118.png")

---

# 🔍 Key Questions to Answer

After running this notebook, look for:

1. **Convergence**: Does λ̂₁×H* increase toward 13 as N increases?
2. **β₁ pattern**: How does the graph Betti number change with ratio?
3. **Zero modes**: Why does Δ₁ have more zero modes at higher ratio?
4. **Mode structure**: Is the cos(θ) pattern on S¹ stronger at ratio=1.18?
5. **Transition**: Is there a sharp transition at ratio ≈ 1.3?